In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import operator
from functools import reduce

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, recall_score


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import os 
os.chdir('/Users/deepak/capstone/clutch-baller')
# os.chdir('./clutch-baller')
import practice_code as pc
import indiviual_stats as ind
# os.chdir('..')

In [2]:
df = pd.read_csv('/Users/deepak/Desktop/NBA_Data/Every_Season_Data/[2005-2006]-combined-stats.csv')

##Added another column that shows the pts difference between both teams

In [3]:
new_df = pc.add_columns(df)

/Users/deepak/capstone/clutch-baller/practice_code.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2['winner'] = np.where(df2.pts_difference >0, 'away', 'home')


In [4]:
new_df = pc.replace_nan(new_df)

##Remove unnecessary columns

In [5]:
col_to_remove = ['a1', 'a2', 'a3','a4','a5','h1','h2','h3','h4','h5', 'play_length', 'entered', 'left', 'possession', 'shot_distance', 'original_x', 'original_y', 'converted_x', 'converted_y', 'num', 'away', 'home', 'outof', 'opponent', 'reason', 'elapsed', 'play_id']

In [6]:
df_1 = pc.remove_col(new_df, col_to_remove)

##Remove Rows that are not needed (such as fouls, subs, timeouts etc)

In [7]:
df2 = pc.remove_junk_rows(df_1, 'event_type')

#Make DF for all players stats

In [8]:
#x = df2[df2['description'].str.contains('Shot') | df2['description'].str.contains('Fadeaway')]# - (df2['description'].str.contains('3PT')*1)



In [9]:
#x = (((new_df['points'] != 3) & new_df['type'].str.contains('Shot') |(new_df['points'] != 3) & new_df['type'].str.contains('Fadeaway')*1).sum())




In [10]:
#y = new_df[(new_df['type'].str.contains('Layup') | new_df['type'].str.contains('Dunk') | new_df['type'].str.contains('Finger Roll'))]




In [11]:
#x = new_df[((new_df['event_type']=='miss') | (new_df['event_type']=='shot'))]

In [12]:
player_stats = ind.object_to_int(df2, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [13]:
assists = ind.assist_stats(player_stats)

In [14]:
blks = ind.blk_stats(player_stats)

In [15]:
names = ind.player_names(player_stats)

In [16]:
full_player_stats = ind.merge_all(names, blks, assists, player_stats)

In [18]:
full_player_stats = full_player_stats.drop(0)

##Regular Season or playoffs (using df_2)

In [324]:
regular_season, playoffs = pc.regular_or_playoffs(df2, 'data_set')

##Getting clutch games

In [156]:
regular_clutch = pc.clutch_moment(regular_season)

In [157]:
regular_clutch = pc.remove_not_imp_games(regular_clutch, 'game_id')

##Stats for all players in reg season, playoff, and clutch

In [217]:
player_stats_clutch = ind.object_to_int(regular_clutch, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [218]:
assists = ind.assist_stats(player_stats_clutch)

In [219]:
blks = ind.blk_stats(player_stats_clutch)

In [220]:

names = ind.player_names(player_stats_clutch)


In [221]:
full_clutch_stats = ind.merge_all(names, blks, assists, player_stats_clutch)

In [222]:
full_clutch_stats = ind.get_per_game_stats(full_clutch_stats)

In [223]:
ids = ind.add_ids(df2)

In [224]:
full_clutch_stats = ind.merge_ids(full_clutch_stats, ids)

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x_df = new_df[new_df['total_shots']>9]


In [225]:
full_clutch_stats = full_clutch_stats.fillna(0)

In [227]:
full_clutch_stats['league_clutch_all_shot_avg'] = round((full_clutch_stats['shots_made'].sum())/(full_clutch_stats['total_shots'].sum())*100,1)
full_clutch_stats['league_clutch_2pt_avg'] = round((full_clutch_stats['2pt_fg_made'].sum())/(full_clutch_stats['2fg_attempts'].sum())*100,1)
full_clutch_stats['league_clutch_3pt_avg'] = round((full_clutch_stats['3pt'].sum())/(full_clutch_stats['3pt_shots'].sum())*100,1)
full_clutch_stats['league_clutch_hard2pt_avg'] = round((full_clutch_stats['2pt_med/hard'].sum())/(full_clutch_stats['med/hard_attempts'].sum())*100,1)
full_clutch_stats['league_clutch_easy2pt_avg'] = round((full_clutch_stats['Dunk/Layup'].sum())/(full_clutch_stats['Dunk/Layup_attempts'].sum())*100,1)






In [228]:
full_clutch_stats['clutch_all_shots'] = (full_clutch_stats['shooting%'] > full_clutch_stats['league_clutch_all_shot_avg']) & (full_clutch_stats['total_shots']>15)
full_clutch_stats['clutch_2pt'] = (full_clutch_stats['2pt%'] > full_clutch_stats['league_clutch_2pt_avg']) & (full_clutch_stats['total_shots']>15)
full_clutch_stats['clutch_3pt'] = (full_clutch_stats['3pt%'] > full_clutch_stats['league_clutch_3pt_avg']) & (full_clutch_stats['total_shots']>15)
full_clutch_stats['clutch_hard2'] = (full_clutch_stats['med/hard_fg%'] > full_clutch_stats['league_clutch_hard2pt_avg']) & (full_clutch_stats['total_shots']>15)
full_clutch_stats['clutch_easy2'] = (full_clutch_stats['easy_shot%'] > full_clutch_stats['league_clutch_easy2pt_avg']) & (full_clutch_stats['total_shots']>15)
                                 
                                 
                                 


In [229]:
#Keep!!!!!!
y = full_clutch_stats[(full_clutch_stats['clutch_all_shots']==True) | (full_clutch_stats['clutch_2pt']==True) | (full_clutch_stats['clutch_3pt']==True) | (full_clutch_stats['clutch_hard2']==True) | (full_clutch_stats['clutch_easy2']==True)]


In [230]:
#Keep!!!!
x = full_clutch_stats[(full_clutch_stats['clutch_all_shots']==True) & (full_clutch_stats['clutch_2pt']==True) & (full_clutch_stats['clutch_3pt']==True) & (full_clutch_stats['clutch_hard2']==True) & (full_clutch_stats['clutch_easy2']==True)]



In [212]:
player_stats_reg = ind.object_to_int(regular_season, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [213]:
reg_assists = ind.assist_stats(player_stats_reg)
reg_blks = ind.blk_stats(player_stats_reg)
reg_names = ind.player_names(player_stats_reg)
full_reg_stats = ind.merge_all(reg_names, reg_blks, reg_assists, player_stats_reg)
full_reg_stats = ind.get_per_game_stats(full_reg_stats)
full_reg_stats = ind.merge_ids(full_reg_stats, ids)


/Users/deepak/capstone/clutch-baller/indiviual_stats.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  x_df = new_df[new_df['total_shots']>9]


In [214]:
full_reg_stats = full_reg_stats.fillna(0)

In [215]:
full_reg_stats['league_reg_all_shot_avg'] = round((full_reg_stats['shots_made'].sum())/(full_reg_stats['total_shots'].sum())*100,1)
full_reg_stats['league_reg_2pt_avg'] = round((full_reg_stats['2pt_fg_made'].sum())/(full_reg_stats['2fg_attempts'].sum())*100,1)
full_reg_stats['league_reg_3pt_avg'] = round(((full_reg_stats['3pt'].sum())/(full_reg_stats['3pt_shots'].sum()))*100,1)
full_reg_stats['league_reg_hard2pt_avg'] = round((full_reg_stats['2pt_med/hard'].sum())/(full_reg_stats['med/hard_attempts'].sum())*100,1)
full_reg_stats['league_reg_easy2pt_avg'] = round((full_reg_stats['Dunk/Layup'].sum())/(full_reg_stats['Dunk/Layup_attempts'].sum())*100,1)


In [325]:
player_stats_playoffs = ind.object_to_int(playoffs, 'points')

/Users/deepak/capstone/clutch-baller/indiviual_stats.py:12: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  new_df = df.convert_objects(convert_numeric=True)


In [329]:
playoffs_assists = ind.assist_stats(player_stats_reg)
playoffs_blks = ind.blk_stats(player_stats_reg)
playoffs_names = ind.player_names(player_stats_reg)
full_playoffs = ind.merge_all(reg_names, reg_blks, reg_assists, player_stats_playoffs)
full_playoffs = ind.get_per_game_stats(full_playoffs)
full_playoffs = ind.merge_ids(full_playoffs, ids)

In [331]:
full_playoffs = full_playoffs.fillna(0)

In [332]:

full_playoffs['league_playoffs_all_shot%'] = round((full_playoffs['shots_made'].sum())/(full_playoffs['total_shots'].sum())*100,1)
full_playoffs['league_playoffs_2pt%'] = round((full_playoffs['2pt_fg_made'].sum())/(full_playoffs['2fg_attempts'].sum())*100,1)
full_playoffs['league_playoffs_3pt%'] = round(((full_playoffs['3pt'].sum())/(full_playoffs['3pt_shots'].sum()))*100,1)
full_playoffs['league_playoffs_hard2pt%'] = round((full_playoffs['2pt_med/hard'].sum())/(full_playoffs['med/hard_attempts'].sum())*100,1)
full_playoffs['league_playoffs_easy2pt%'] = round((full_playoffs['Dunk/Layup'].sum())/(full_playoffs['Dunk/Layup_attempts'].sum())*100,1)

In [333]:
full_reg_playoffs.head()

,player,id,id_x,id_y,total_games,shots_made,shots_missed,3pt,3pt_shots,Dunk/Layup,Dunk/Layup_attempts,2pt_med/hard,med/hard_attempts,2fg_attempts,FT_made,FT_missed,assist_count,off_rebound,total_rebound,total_blocks,2pt_fg_made,total_shots,shooting%,FT%,APG,BPG,ORPG,RPG,easy_shot%,2pt%,3pt%,med/hard_fg%,league_playoffs_all_shot_avg,league_playoffs_2pt_avg,league_playoffs_3pt_avg,league_playoffs_hard2pt_avg,league_playoffs_easy2pt_avg
1,Aaron McKie,1,1.0,1.0,0,3,9,0,2,0,0,3,10,10,1,1,11,3,20,0,3,12,25.0,50.0,inf,NaN,inf,inf,NaN,30.0,0.0,30.0,45.4,47.8,35.8,40.3,60.7
2,Aaron Miles,2,2.0,2.0,0,6,12,0,0,2,7,4,11,18,4,0,24,0,14,1,6,18,33.3,100.0,inf,inf,NaN,inf,28.6,33.3,NaN,36.4,45.4,47.8,35.8,40.3,60.7
3,Aaron Williams,3,3.0,3.0,0,91,85,0,0,59,85,32,91,176,40,18,17,64,181,21,91,176,51.7,69.0,inf,inf,inf,inf,69.4,51.7,NaN,35.2,45.4,47.8,35.8,40.3,60.7
4,Adonal Foyle,4,4.0,4.0,0,152,148,0,0,80,122,72,178,300,41,26,33,143,424,125,152,300,50.7,61.2,inf,inf,inf,inf,65.6,50.7,NaN,40.4,45.4,47.8,35.8,40.3,60.7
5,Adrian Griffin,5,5.0,5.0,20,98,106,0,2,36,64,62,138,202,41,12,89,75,227,9,98,204,48.0,77.4,4.400000,0.400000,3.800000,11.400000,56.2,48.5,0.0,44.9,45.4,47.8,35.8,40.3,60.7
